# Demo for Application Scorecard Model Documentation

This interactive notebook guides model developers through the process of documenting a model with the ValidMind Developer Framework. It uses the [Lending Club](https://www.kaggle.com/code/kmalit/bank-customer-churn-prediction/data) sample dataset from ... to build a simple application scorecard.

As part of the notebook, you will learn how to train a sample model while exploring how the documentation process works:

- Initializing the ValidMind Developer Framework
- Loading a sample dataset provided by the library to train an application scorecard model
- Running a ValidMind test suite to quickly generate documention about the data and model

## ValidMind at a glance

ValidMind's platform enables organizations to identify, document, and manage model risks for all types of models, including AI/ML models, LLMs, and statistical models. As a model developer, you use the ValidMind Developer Framework to automate documentation and validation tests, and then use the ValidMind AI Risk Platform UI to collaborate on model documentation. Together, these products simplify model risk management, facilitate compliance with regulations and institutional standards, and enhance collaboration between yourself and model validators.

If this is your first time trying out ValidMind, you can make use of the following resources alongside this notebook:

- [Get started](https://docs.validmind.ai/guide/get-started.html) — The basics, including key concepts, and how our products work
- [Get started with the ValidMind Developer Framework](https://docs.validmind.ai/guide/get-started-developer-framework.html) — The path for developers, more code samples, and our developer reference


## Before you begin

::: {.callout-tip}

### New to ValidMind?

For access to all features available in this notebook, create a free ValidMind account.

Signing up is FREE — [**Sign up now**](https://app.prod.validmind.ai)
:::

If you encounter errors due to missing modules in your Python environment, install the modules with `pip install`, and then re-run the notebook. For more help, refer to [Installing Python Modules](https://docs.python.org/3/installing/index.html).

## Install the client library

The client library provides Python support for the ValidMind Developer Framework. To install it:


In [ ]:
%pip install -q validmind

## Initialize the client library

ValidMind generates a unique _code snippet_ for each registered model to connect with your developer environment. You initialize the client library with this code snippet, which ensures that your documentation and tests are uploaded to the correct model when you run the notebook.

Get your code snippet:

1. In a browser, log into the [Platform UI](https://app.prod.validmind.ai).

2. In the left sidebar, navigate to **Model Inventory** and click **+ Register new model**.

3. Enter the model details and click **Continue**. ([Need more help?](https://docs.validmind.ai/guide/register-models-in-model-inventory.html))

   For example, to register a model for use with this notebook, select:

   - Documentation template: `Binary classification`
   - Use case: `Marketing/Sales - Attrition/Churn Management`

   You can fill in other options according to your preference.

4. Go to **Getting Started** and click **Copy snippet to clipboard**.

Next, replace this placeholder with your own code snippet:

In [ ]:
import validmind as vm

vm.init(
  api_host = "https://api.prod.validmind.ai/api/v1/tracking",
  api_key = "...",
  api_secret = "...",
  project = "..."
)

## Initialize the Python environment

Next, let's import the necessary libraries and set up your Python environment for data analysis:

In [ ]:
import statsmodels.api as sm

%matplotlib inline

### Preview the documentation template

A template predefines sections for your model documentation and provides a general outline to follow, making the documentation process much easier.

You will upload documentation and test results into this template later on. For now, take a look at the structure that the template provides with the `vm.preview_template()` function from the ValidMind library and note the empty sections:

In [ ]:
vm.preview_template()

## Load the sample dataset

The sample dataset used here is provided by the ValidMind library. To be able to use it, you need to import the dataset and load it into a pandas [DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html), a two-dimensional tabular data structure that makes use of rows and columns:

In [ ]:
# Import the sample dataset from the library

from validmind.datasets.credit_risk import lending_club

df = lending_club.load_data(source="offline")

df.head()

### Prepocess the dataset

Preprocessing performs a number of operations to get ready for the subsequent steps:

- Preprocess the data: Splits the DataFrame (`df`) into multiple datasets (`train_df`, `validation_df`, and `test_df`) using `lending_club.preprocess` to simplify preprocessing.

In [ ]:
preprocess_df = lending_club.preprocess(df)
preprocess_df.head()

### Feature engineering

TBC.

In [ ]:
fe_df = lending_club.feature_engineering(preprocess_df)
fe_df.head()

## Train the model

In this section, we focus on constructing and refining our predictive model. Initially, we divide our data, which is based on Weight of Evidence (WoE) features, into training and testing sets (`train_df`, `test_df`). With `lending_club.split`, we employ a simple random split, randomly allocating data points to each set to ensure a mix of examples in both. Additionally, by setting `add_constant=True`, we include an intercept term in our model.



In [ ]:
train_df, test_df = lending_club.split(fe_df, add_constant=True)

x_train = train_df.drop(lending_club.target_column, axis=1)
y_train = train_df[lending_club.target_column]
x_test = test_df.drop(lending_club.target_column, axis=1)
y_test = test_df[lending_club.target_column]

# Define the model
model = sm.GLM(
    y_train, 
    x_train, 
    family=sm.families.Binomial())

# Fit the model
model = model.fit()
model.summary()

### Compute Scores

In this phase, we translate model predictions into actionable scores using probability estimates generated by our trained model. 

In [ ]:
# Compute probabilities from the model
train_probabilities = model.predict(x_train)
test_probabilities = model.predict(x_test)

# Compute scores from the predictions
train_scores = lending_club.compute_scores(train_probabilities)
test_scores = lending_club.compute_scores(test_probabilities)

## Document the model

As part of documenting the model with the ValidMind Developer Framework, you need to preprocess the raw dataset, initialize some training and test datasets, initialize a model object you can use for testing, and then run the full suite of tests.

### Initialize the ValidMind datasets

Before you can run tests, you must first initialize a ValidMind dataset object using the [`init_dataset`](https://docs.validmind.ai/validmind/validmind.html#init_dataset) function from the ValidMind (`vm`) module.

This function takes a number of arguments:

- `dataset` — the dataset that you want to provide as input to tests
- `input_id` - a unique identifier that allows tracking what inputs are used when running each individual test
- `target_column` — a required argument if tests require access to true values. This is the name of the target column in the dataset

With all datasets ready, you can now initialize the raw, processed, training and test datasets (`raw_df`, `clean_df`, `preprocessed_df`, `fe_df`,  `train_df` and `test_df`) created earlier into their own dataset objects using [`vm.init_dataset()`](https://docs.validmind.ai/validmind/validmind.html#init_dataset):

In [ ]:
vm_raw_dataset = vm.init_dataset(
    dataset=df,
    input_id="raw_dataset",
    target_column=lending_club.target_column,
)

vm_preprocess_dataset = vm.init_dataset(
    dataset=preprocess_df,
    input_id="preprocess_dataset",
    target_column=lending_club.target_column,
)

vm_fe_dataset = vm.init_dataset(
    dataset=fe_df,
    input_id="fe_dataset",
    target_column=lending_club.target_column,
)

vm_train_ds = vm.init_dataset(
    dataset=train_df,
    input_id="train_dataset",
    target_column=lending_club.target_column,
)

vm_test_ds = vm.init_dataset(
    dataset=test_df, 
    input_id="test_dataset", 
    target_column=lending_club.target_column
)

TBC.

In [ ]:
print(vm_raw_dataset)
print(vm_preprocess_dataset)
print(vm_fe_dataset)
print(vm_train_ds)
print(vm_test_ds)

### Initialize a model object

Additionally, you need to initialize a ValidMind model object (`vm_model`) that can be passed to other functions for analysis and tests on the data. You simply intialize this model object with [`vm.init_model()`](https://docs.validmind.ai/validmind/validmind.html#init_model):

In [ ]:
vm_model = vm.init_model(
    model,
    input_id="glm_model",
)

### Assign predictions to the datasets

With our model now trained, the next step involves assigning its predictions to our datasets. This is achieved through the use of the `assign_predictions()` method associated with the VM `dataset` object. This method facilitates the linkage of the model's prediction outcomes to the corresponding datasets. It is designed to accept explicitly provided prediction values; however, in their absence, it will automatically compute the predictions.

In [ ]:
vm_train_ds.assign_predictions(
    model=vm_model,
    prediction_values=train_probabilities,
)

vm_test_ds.assign_predictions(
    model=vm_model,
    prediction_values=test_probabilities,
)

### Assign scores to the datasets

In the context of credit scorecards, we are interested in computing scores from model predictions, as scorecards fundamentally revolve around these scores rather than raw predictions. To facilitate this process and ensure scores are properly integrated with our datasets, we use the `add_extra_column()` method from the VM `dataset` object. This approach allows us to append scores directly to our data, maintaining a streamlined and coherent dataset ready for analysis.

In [ ]:
vm_train_ds.add_extra_column("glm_scores", train_scores)
vm_test_ds.add_extra_column("glm_scores", test_scores)

print(vm_train_ds)
print(vm_test_ds)

### Data validation

Data validation is our quality checkpoint before diving into the depths of analysis. Here, we make sure that the data we're working with is accurate, consistent, and ready for rigorous testing. We're looking for anything out of the ordinary that might skew our models or results.

#### Run tests for raw data tests

Initial scrutiny is applied to the unprocessed data to detect any anomalies. This involves:

- *Tabular Description Tables*: Offer a statistical summary for each variable.
- *Missing Values Bar Plot*: Chart out the presence of any missing data.

In [ ]:

test= vm.tests.run_test(
    "validmind.data_validation.TabularDescriptionTables",
    inputs = {
        "dataset": vm_raw_dataset
    }
)
test.log()

In [ ]:

test= vm.tests.run_test(
    "validmind.data_validation.MissingValuesBarPlot",
    inputs = {
        "dataset": vm_raw_dataset
    }
)
test.log()

#### Run tests for preprocessed data

Prior to modeling, the data undergoes a final review to affirm its preparedness. We conduct:

- *Tabular Description Tables*: Checking structural integrity after preprocessing.
- *IQR Outliers Table*: Searching for statistical anomalies that may impact results.
- *Class Imbalance*: Investigating the proportionality of outcome classes.
- *Tabular Numerical Histograms*: Visualizing distributions for numerical variables.
- *Tabular Categorical Bar Plots*: Analyzing frequencies of categorical variables.
- *Target Rate Bar Plots*: Examining the distribution of the target variable across categories.
- *Pearson Correlation Matrix*: Identifying linear relationships between variables.


In [ ]:

test= vm.tests.run_test(
    "validmind.data_validation.TabularDescriptionTables",
    inputs = {
        "dataset": vm_preprocess_dataset
    }
)
test.log()

In [ ]:

test= vm.tests.run_test(
    "validmind.data_validation.IQROutliersTable",
    inputs = {
        "dataset": vm_preprocess_dataset
    }
)
test.log()

In [ ]:

test= vm.tests.run_test(
    "validmind.data_validation.ClassImbalance",
    inputs = {
        "dataset": vm_preprocess_dataset
    }
)
test.log()

In [ ]:

test= vm.tests.run_test(
    "validmind.data_validation.TabularNumericalHistograms",
    inputs = {
        "dataset": vm_preprocess_dataset
    }
)
test.log()

In [ ]:

test= vm.tests.run_test(
    "validmind.data_validation.TabularCategoricalBarPlots",
    inputs = {
        "dataset": vm_preprocess_dataset
    }
)
test.log()

In [ ]:

test= vm.tests.run_test(
    "validmind.data_validation.TargetRateBarPlots",
    inputs = {
        "dataset": vm_preprocess_dataset
    },
    params = {
        "default_column": lending_club.target_column,
        "columns": None
    }
)
test.log()

In [ ]:

test= vm.tests.run_test(
    "validmind.data_validation.PearsonCorrelationMatrix",
    inputs = {
        "dataset": vm_preprocess_dataset
    }
)
test.log()

#### Run tests for WoE analysis

Ensuring data segmentation aligns with predictive value. We look into:

- *WOE Bin Table*: Checking the bins and their corresponding Weight of Evidence values.
- *WOE Bin Plots*: Displaying the distribution of WoE across the data spectrum.

In [ ]:

test= vm.tests.run_test(
    "validmind.data_validation.WOEBinTable",
    inputs = {
        "dataset": vm_preprocess_dataset
    },
    params = {
        "breaks_adj": lending_club.breaks_adj
    }
)
test.log()

In [ ]:

test= vm.tests.run_test(
    "validmind.data_validation.WOEBinPlots",
    inputs = {
        "dataset": vm_preprocess_dataset
    },
    params = {
        "breaks_adj": lending_club.breaks_adj
    }
)
test.log()

### Model validation

This phase verifies the model's predictive power and generalizability. Tests are applied to both training and evaluation stages to ensure robustness and accuracy.

#### Run tests for model training

This step assesses the model's learning process for stability and significance. This involves:

- *Regression Coeffs Plot*: Visual inspection of the model's coefficients.
- *Regression Models Coeffs*: Detailed evaluation of the regression coefficients' values.

In [ ]:

test= vm.tests.run_test(
    "validmind.model_validation.statsmodels.RegressionCoeffsPlot",
    inputs = {
        "models": [vm_model]
    }
)
test.log()

In [ ]:

test= vm.tests.run_test(
    "validmind.model_validation.statsmodels.RegressionModelsCoeffs",
    inputs = {
        "models": [vm_model]
    }
)
test.log()

#### Run tests for model evaluation

We measure the model's performance on unseen data to validate its predictive power. We conduct:

- *Log Reg Performance Table*: Summarizing logistic regression metrics.
- *GINI Table*: Assessing the discriminatory power of the model.
- *Log Reg Confusion Matrix*: Understanding classification accuracy.
- *Log Reg ROC Curve*: Plotting the trade-off between sensitivity and specificity.
- *Log Reg Prediction Histogram*: Distributing predictions to visualize outcomes.
- *Log Reg Cumulative Prob*: Cumulative probability analysis for predictions.
- *Scorecard Histogram*: Evaluating the distribution of scorecard points.

In [ ]:

test = vm.tests.run_test(
    "validmind.model_validation.statsmodels.LogRegPerformanceTable",
    inputs={
        "datasets": [vm_train_ds, vm_test_ds],
        "model": vm_model,
    },
    params = {
        "cut_off_threshold": 0.5
    }
)
test.log()

In [ ]:

test= vm.tests.run_test(
    "validmind.model_validation.statsmodels.GINITable",
    inputs = {
        "model": vm_model,
        "datasets": [vm_train_ds, vm_test_ds]
    }
)
test.log()

In [ ]:

test= vm.tests.run_test(
    "validmind.model_validation.statsmodels.LogRegConfusionMatrix",
    inputs = {
        "dataset": vm_test_ds,
        "model": vm_model
    },
    params = {
        "cut_off_threshold": 0.5
    }
)
test.log()

In [ ]:

test= vm.tests.run_test(
    "validmind.model_validation.statsmodels.LogRegROCCurve",
    inputs = {
        "model": vm_model,
        "dataset": vm_test_ds
    }
)
test.log()

In [ ]:

test= vm.tests.run_test(
    "validmind.model_validation.statsmodels.LogRegPredictionHistogram",
    inputs = {
        "model": vm_model,
        "datasets": [vm_train_ds, vm_test_ds]
    }
)
test.log()

In [ ]:

test= vm.tests.run_test(
    "validmind.model_validation.statsmodels.LogRegCumulativeProb",
    inputs = {
        "model": vm_model,
        "datasets": [vm_train_ds, vm_test_ds]
    }
)
test.log()

In [ ]:

test= vm.tests.run_test(
    "validmind.model_validation.statsmodels.ScorecardHistogram",
    inputs = {
        "model": vm_model,
        "datasets": [vm_train_ds, vm_test_ds]
    },
    params = {
        "score_column": "glm_scores"
    }
)
test.log()

## Next steps

You can look at the results of this test suite right in the notebook where you ran the code, as you would expect. But there is a better way: view the test results as part of your model documentation right in the ValidMind Platform UI: 

1. In the [Platform UI](https://app.prod.validmind.ai), go to the **Documentation** page for the model you registered earlier.

2. TBC

What you can see now is a more easily consumable version of the tests you just executed, along with other parts of your model documentation that still need to be completed. 

If you want to learn more about where you are in the model documentation process, take a look at [How do I use the framework?](https://docs.validmind.ai/guide/get-started-developer-framework.html#how-do-i-use-the-framework).